In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import paramiko
from pathlib import Path

os.getcwd()

'/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne'

In [2]:
# To update here 
new_campaing_name = 'Offer_10_off_20_20Q4/'
local_folder = '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/' + new_campaing_name


print("local_folder: \n", local_folder)

try:
    os.stat(local_folder)
except:
    os.mkdir(local_folder)

local_folder: 
 /mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/


In [3]:
host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

username = "agilone" #hard-coded
password = "Tur87ZqzF9xV5mYF" #hard-coded
transport.connect(username=username, password=password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [4]:
remote_folder = "/home/agilone/"
list_remote_files = [remote_folder+x for x in sftp.listdir(remote_folder)]

list_remote_files = [x for x in list_remote_files
                     if pd.to_datetime(sftp.stat(x).st_mtime,unit="s")>=datetime.datetime.now()-datetime.timedelta(days=1)]
list_remote_files = [x for x in list_remote_files if x[-4]=="."]
list_remote_files

['/home/agilone/Near_Lapsing_Delta__20201119021243.csv',
 '/home/agilone/13_Month_Lapsed_Delta__20201119021501.csv',
 '/home/agilone/Signup_NoPurchase_Delta__20201119021251.csv',
 '/home/agilone/Reactivation_Redeemers__20201119031841.csv']

In [5]:
# Remove the ccpa received on the same day
list_remote_files = [x for x in list_remote_files if "ccpa_dele" not in x.lower()]
list_remote_files

['/home/agilone/Near_Lapsing_Delta__20201119021243.csv',
 '/home/agilone/13_Month_Lapsed_Delta__20201119021501.csv',
 '/home/agilone/Signup_NoPurchase_Delta__20201119021251.csv',
 '/home/agilone/Reactivation_Redeemers__20201119031841.csv']

In [6]:
# transfer the files to local server
for file in list_remote_files:
    basename = os.path.basename(file)
    local_path = local_folder + basename
    sftp.get(file, local_path)
sftp.close()
transport.close()

In [18]:
# list out paths for new files in local server
files = [Path(p).name for p in list_remote_files]

files_local_paths = [next(Path(local_folder).glob(f)) for f in files]
files_local_paths

[PosixPath('/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/Near_Lapsing_Delta__20201119021243.csv'),
 PosixPath('/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/13_Month_Lapsed_Delta__20201119021501.csv'),
 PosixPath('/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/Signup_NoPurchase_Delta__20201119021251.csv'),
 PosixPath('/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/Reactivation_Redeemers__20201119031841.csv')]

In [7]:
# import glob
# list_local_file_new=glob.glob(local_folder+"*.csv")
# list_local_file_new.sort()
# list_local_file_new

['/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/10-12 Month About to Lapse MS A_20201102075436.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/10-12 Month About to Lapse MS B_20201102075837.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/13-18 Month Lapsed MS A_20201102073523.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/13-18 Month Lapsed MS B_20201102073920.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/13_Month_Lapsed_Delta__20201119021501.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/Near_Lapsing_Delta__20201119021243.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off_20_20Q4/Online Signup MS A_20201102074635.csv',
 '/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/Offer_10_off

In [24]:
# Manually QC
df = pd.read_csv(files_local_paths[3], nrows=10)
df

,customersummary_mastercustomerid,customersummary_emailhash,customersummary_c_primaryscnhash,execution_id
0,KFK_0_8080278,926a7e8ae937982587102acdd9f8be79999f7b9dfe8674...,a1c99d904e9801648c3c7e4f43a64caec2feb4cf3e8d04...,1605796990823
1,FTP_CSV_6_42733719,9628d15eb4665dd40d1766d429dbe19d6b2655858de9d8...,ecdce8d80cf2365546169913aafe6ff7033ac1405f3b27...,1605796990823
2,KFK_0_58501153,46c2136ea20df6c6126b45e754a9ee10037496667b3574...,e232f00c8de9ef81e5a87cac82c82b4787517bd3211b2a...,1605796990823
3,KFK_0_59424453,28451578c9f1234c5c52b76c0b7d2578c72b8ea4d0e066...,b4cd2c350506726b5009caf3926579f799dc8d4a28dd55...,1605796990823
4,KFK_0_37221152,cf8ed7de337396945c6fc66f5eb6f96103a4918edf7ce1...,ca6965083c5ecb76cabee74e04f5ac27a659b2fdb20cd8...,1605796990823
5,KFK_0_52323482,0e7c2056b8cdd1b0f03c735dbcf93744f7d8f66da4c7ad...,0665aa38725315f66a7349bd39988e62ee093b7f527d81...,1605796990823
6,KFK_0_51468806,16b3e74a61d931d4a79ff18bf1446cd20a6346d9fc2b91...,05eae8d3527e030628b92f826b8da35fd631370e5dcf34...,1605796990823
7,KFK_0_61402364,41c31e9ad390affe3af670baed97fb98e0b588462f5831...,df248233039f781dd0b08ad572286dbaaf5eeed8fb3ee8...,1605796990823
8,KFK_0_58783951,b43667864138db81f85027ebf43e087c42975b453af5dd...,1a0b9f4437e968ac81edd50c1ff8bfa586ecbed4955f3b...,1605796990823
9,KFK_0_58599901,b77864808cd354019d0e03e3caddea4df5506f8ef129dc...,240698e6d7e1137a0974482b1ffe7add66b1b4931391d0...,1605796990823


In [20]:
df_summary_count = pd.DataFrame()
i = 0
for file in files_local_paths:
    i += 1
    df = pd.read_csv(file, dtype=str)
    # name, row, unique_id, unique_email
    
    basename = os.path.basename(file)
    row_count = df.shape[0]
    unique_id = df['customersummary_c_primaryscnhash'].nunique()
    unique_email = df['customersummary_emailhash'].nunique()
    
    print(basename,row_count,unique_id,unique_email)
    print("All sha256 format in 64 length: ",
          df['customersummary_emailhash'].apply(len).unique())
    df_s_file = pd.DataFrame({"file": basename,
                              "row_count": row_count,
                              "unique_id": unique_id,
                              "unique_email": unique_email}, index=[i])
    df_summary_count = df_summary_count.append(df_s_file)
    if df['customersummary_emailhash'].apply(len).unique().tolist() != [64]:
        print("Checking wrong length (remember to remove when uploading): ")
        df['email_len']=df['customersummary_emailhash'].apply(len)
        df=df.groupby("email_len")['customersummary_emailhash'].count().to_frame().reset_index()
        print(df.set_index("email_len").to_dict()['customersummary_emailhash'])
        
        

Near_Lapsing_Delta__20201119021243.csv 148705 148705 148704
All sha256 format in 64 length:  [64]
13_Month_Lapsed_Delta__20201119021501.csv 164099 164099 164098
All sha256 format in 64 length:  [64]
Signup_NoPurchase_Delta__20201119021251.csv 172337 172337 172330
All sha256 format in 64 length:  [64  7]
Checking wrong length (remember to remove when uploading): 
{7: 8, 64: 172329}
Reactivation_Redeemers__20201119031841.csv 19793 19792 19792
All sha256 format in 64 length:  [64  7 12]
Checking wrong length (remember to remove when uploading): 
{7: 2, 12: 1, 64: 19790}


In [21]:
df_summary_count

,file,row_count,unique_id,unique_email
1,Near_Lapsing_Delta__20201119021243.csv,148705,148705,148704
2,13_Month_Lapsed_Delta__20201119021501.csv,164099,164099,164098
3,Signup_NoPurchase_Delta__20201119021251.csv,172337,172337,172330
4,Reactivation_Redeemers__20201119031841.csv,19793,19792,19792
